In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=7258dfd56681c1dbd6fa934e6f513da633ed6083ed9247b66abe80d7093a9ce0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


# PySpark - Modelo de Regressão
- Regressão Linear
- Random Forest

In [6]:
from pyspark.sql.session import SparkSession

from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.feature import VectorAssembler

## Criando a Session PySpark

In [7]:
spark = SparkSession.builder.appName('regressaolinear_randomforest')\
        .config('spark.master', 'local')\
        .config('spark.executor.memory', '2gb')\
        .config('spark.shuffle.sql.partitions', 2)\
        .getOrCreate()

## Importando a base

In [68]:
carros_temp = spark.read.csv(
    '/content/Carros.csv',
    inferSchema = True,
    header = True,
    sep = ';')

In [69]:
# verificando os primeiros registros
carros_temp.show(5)

+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|Consumo|Cilindros|Cilindradas|RelEixoTraseiro|Peso|Tempo|TipoMotor|Transmissao|Marchas|Carburadors| HP|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
|     21|        6|        160|             39| 262| 1646|        0|          1|      4|          4|110|
|     21|        6|        160|             39|2875| 1702|        0|          1|      4|          4|110|
|    228|        4|        108|            385| 232| 1861|        1|          1|      4|          1| 93|
|    214|        6|        258|            308|3215| 1944|        1|          0|      3|          1|110|
|    187|        8|        360|            315| 344| 1702|        0|          0|      3|          2|175|
+-------+---------+-----------+---------------+----+-----+---------+-----------+-------+-----------+---+
only showing top 5 rows



In [70]:
carros_temp.count()

32

In [71]:
# verificando as informações de cada variável
carros_temp.printSchema()

root
 |-- Consumo: integer (nullable = true)
 |-- Cilindros: integer (nullable = true)
 |-- Cilindradas: integer (nullable = true)
 |-- RelEixoTraseiro: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Tempo: integer (nullable = true)
 |-- TipoMotor: integer (nullable = true)
 |-- Transmissao: integer (nullable = true)
 |-- Marchas: integer (nullable = true)
 |-- Carburadors: integer (nullable = true)
 |-- HP: integer (nullable = true)



In [72]:
# criando um novo df
carros = carros_temp.select('Consumo', 'Cilindros', 'Cilindradas', 'HP')

In [73]:
carros.show(5)

+-------+---------+-----------+---+
|Consumo|Cilindros|Cilindradas| HP|
+-------+---------+-----------+---+
|     21|        6|        160|110|
|     21|        6|        160|110|
|    228|        4|        108| 93|
|    214|        6|        258|110|
|    187|        8|        360|175|
+-------+---------+-----------+---+
only showing top 5 rows



## VectorAssembler
- - Cria uma nova coluna do tipo lista com os de cada linha


In [74]:
# Criando um objeto VectorAssebler
vec_caracteristicas = VectorAssembler(inputCols = ['Consumo', 'Cilindros', 'Cilindradas'],
                                      outputCol = 'caracteristicas')

In [75]:
# transform para transformar as variáveis
carros = vec_caracteristicas.transform(carros)

In [76]:
carros.show(5)

+-------+---------+-----------+---+-----------------+
|Consumo|Cilindros|Cilindradas| HP|  caracteristicas|
+-------+---------+-----------+---+-----------------+
|     21|        6|        160|110| [21.0,6.0,160.0]|
|     21|        6|        160|110| [21.0,6.0,160.0]|
|    228|        4|        108| 93|[228.0,4.0,108.0]|
|    214|        6|        258|110|[214.0,6.0,258.0]|
|    187|        8|        360|175|[187.0,8.0,360.0]|
+-------+---------+-----------+---+-----------------+
only showing top 5 rows



## Dividindo os dados em treino e teste
- Treino: 70% da base de dados
- Teste: 30% da base de dados

In [77]:
# realização das divisões
carros_treino, carros_teste = carros.randomSplit([0.7, 0.3])

In [78]:
print(carros_treino.count())
print(carros_teste.count())

26
6


## Regressão Linear

#### Criando o objeto LinearRegression
- featuresCol = entradas
- labelCol = saida (Dado que queremos prever)

In [79]:
rl = LinearRegression(featuresCol = 'caracteristicas', labelCol = 'HP')

#### Criando o modelo da Regressão Linear
- .fit -> para treinar os dados de treino (70%)

In [80]:
model_rl = rl.fit(carros_treino)

#### Previsão do modelo
- Precisa passar os dados de teste (30%)
- Retorna um novo df com uma coluna chamada 'prediction'

In [81]:
previsao = model_rl.transform(carros_teste)

#### Entendendo as previsões
- Dados reais (HP)
- Dados previstos (prediction)
- Na primeira linha, o dado real do HP é 335 e  o modelo previu na 184

In [82]:
previsao.show()

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     26|        4|       1203| 91| [26.0,4.0,1203.0]| 91.37031469344534|
|    152|        8|        304|150| [152.0,8.0,304.0]|208.70412096994474|
|    158|        8|        351|264| [158.0,8.0,351.0]|207.77775952185848|
|    173|        8|       2758|180|[173.0,8.0,2758.0]| 187.4982886033123|
|    214|        4|        121|109| [214.0,4.0,121.0]| 82.38840502574772|
|    214|        6|        258|110| [214.0,6.0,258.0]|142.30835247036757|
+-------+---------+-----------+---+------------------+------------------+



## Avaliando a performance do modelo da Regressão Linear

#### Criando o objeto de avaliação da regressão
- RegressionEvaluator()
    - 1º Parametro (<b>predictionCol</b>):
        - A coluna (prediciton) que o modelo previu.
    - 2º Parametro (<b>labelCol</b>):
        - A coluna (HP) com os dados reais que o modelo utilizou para prever.
    - 2º Parametro (<b>metricName</b>):
        - Nome da métrica
    - rmse: quanto menor, melhor

In [83]:
avaliar = RegressionEvaluator(predictionCol = 'prediction',
                              labelCol = 'HP',
                              metricName = 'rmse')

#### Avaliando o modelo

In [84]:
rmse = avaliar.evaluate(previsao)

In [85]:
print(rmse)

37.45103967094857


## Random Forest

#### Criando o objeto LinearRegression
- featuresCol = entradas
- labelCol = saida (Dado que queremos prever)

In [86]:
rf = RandomForestRegressor(featuresCol = 'caracteristicas',
                           labelCol = 'HP')

#### Criando o modelo da Regressão Linear
- .fit -> para treinar os dados de treino (70%)

In [87]:
model_rf = rf.fit(carros_treino)

#### Previsão do modelo
- Precisa passar os dados de teste (30%)
- Retorna um novo df com uma coluna chamada 'prediction'

In [88]:
previsao_rf = model_rf.transform(carros_teste)

#### Entendendo as previsões
- Dados reais (HP)
- Dados previstos (prediction)
- Na primeira linha, o dado real do HP é 335 e o modelo previu na 177

In [89]:
previsao_rf.show(5)

+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     26|        4|       1203| 91| [26.0,4.0,1203.0]|            114.79|
|    152|        8|        304|150| [152.0,8.0,304.0]| 243.8136188087275|
|    158|        8|        351|264| [158.0,8.0,351.0]|209.21540452301323|
|    173|        8|       2758|180|[173.0,8.0,2758.0]| 155.4422360248447|
|    214|        4|        121|109| [214.0,4.0,121.0]| 89.55833333333334|
+-------+---------+-----------+---+------------------+------------------+
only showing top 5 rows



## Avaliando a performance do modelo da Regressão Linear

In [90]:
rmse_rf = avaliar.evaluate(previsao_rf)

In [91]:
print(rmse_rf)

47.1992144009642


## Comparando os modelos

In [92]:
import pandas as pd
pd.DataFrame([rmse, rmse_rf], ['rmse_lr', 'rmse_rf'], columns = ['Avaliação de Desempenho'])

,Avaliação de Desempenho
rmse_lr,37.451040
rmse_rf,47.199214


- LinearRegression (rmse)

In [93]:
# rmse -> LinearRegression
print(f'rmse: {rmse:.2f}%')
previsao.show()

rmse: 37.45%
+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     26|        4|       1203| 91| [26.0,4.0,1203.0]| 91.37031469344534|
|    152|        8|        304|150| [152.0,8.0,304.0]|208.70412096994474|
|    158|        8|        351|264| [158.0,8.0,351.0]|207.77775952185848|
|    173|        8|       2758|180|[173.0,8.0,2758.0]| 187.4982886033123|
|    214|        4|        121|109| [214.0,4.0,121.0]| 82.38840502574772|
|    214|        6|        258|110| [214.0,6.0,258.0]|142.30835247036757|
+-------+---------+-----------+---+------------------+------------------+



- RandomForestRegressor (rmse)

In [94]:
# rmse -> RandomForestRegressor
print(f'rmse: {rmse_rf:.2f}%')
previsao_rf.show()

rmse: 47.20%
+-------+---------+-----------+---+------------------+------------------+
|Consumo|Cilindros|Cilindradas| HP|   caracteristicas|        prediction|
+-------+---------+-----------+---+------------------+------------------+
|     26|        4|       1203| 91| [26.0,4.0,1203.0]|            114.79|
|    152|        8|        304|150| [152.0,8.0,304.0]| 243.8136188087275|
|    158|        8|        351|264| [158.0,8.0,351.0]|209.21540452301323|
|    173|        8|       2758|180|[173.0,8.0,2758.0]| 155.4422360248447|
|    214|        4|        121|109| [214.0,4.0,121.0]| 89.55833333333334|
|    214|        6|        258|110| [214.0,6.0,258.0]|114.14991452991453|
+-------+---------+-----------+---+------------------+------------------+

